In [2]:
import os
os.environ["CALITP_BQ_MAX_BYTES"] = str(800_000_000_000)
import shared_utils

import geopandas as gpd
import pandas as pd
import folium 
import branca
import datetime as dt
from siuba import *
from bus_service_utils import better_bus_utils

from rt_analysis import rt_filter_map_plot

ZERO_THIRTY_COLORSCALE = branca.colormap.step.RdYlGn_11.scale(vmin=0, vmax=35)

FileNotFoundError: calitp-analytics-data/data-analyses/ca_county_centroids.parquet

In [ ]:
import bb_rt_helper

In [ ]:
import importlib
importlib.reload(bb_rt_helper)

# 100 Recs - D6

## 1 - Corridor Improvements

In [ ]:
bb_rt_helper.fm_from_bbutils?

In [ ]:
fm_dict = bb_rt_helper.fm_from_bbutils(ct_dist='06 - Fresno', category='corridor')

In [ ]:
corr_speed_views = []
for key in fm_dict.keys():
    _m = fm_dict[key].segment_speed_map()
    corr_speed_views += [fm_dict[key].detailed_map_view.copy()]

### Operator: Madera County Connection

In [ ]:
# manually add route filter via trip ids
mcc_trips = (fm_dict[188].rt_trips
             >> filter(_.route_id=="3615")
            ).trip_id
mcc_trips

In [ ]:
fm_dict[188].rt_trips.route_short_name.isnull().all()

In [ ]:
fm_dict[188].set_filter(trip_ids = mcc_trips)

In [ ]:
m_corr = fm_dict[188].segment_speed_map(no_title = True)

In [ ]:
m_corr

no recommendations from here shrug emoji

### Operator: GET Bus

In [ ]:
m_corr = fm_dict[126].segment_speed_map()

In [ ]:
m_corr

## Draft Corridor Recs, GET Bus, D6

* Route 82/84, Calloway Drive to Fruitvale Ave
* Route 43 along O st + 34th st 

## Route 82

In [ ]:
shape_id = '7'
stop_range = [3, 8]

In [ ]:
fm_dict[126].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[126].reset_filter()

In [ ]:
fm_dict[126].set_filter(route_names=['82'])

In [ ]:
fm_dict[126].corridor_metrics()

In [ ]:
fm_dict[126].quick_map_corridor()

In [ ]:
corr_82 = fm_dict[126].corridor.copy() # important to use .copy() here to avoid referencing the changing corridor

In [ ]:
corridors = [corr_82]

## Route 84

In [ ]:
shape_id = '9'
stop_range = [17, 19]

In [ ]:
fm_dict[126].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[126].set_filter(route_names=['84'])

In [ ]:
fm_dict[126].corridor_metrics()

In [ ]:
fm_dict[126].quick_map_corridor()

In [ ]:
corr_84 = fm_dict[126].corridor.copy()
corridors += [corr_84]

## Route 43

In [ ]:
shape_id = '25'
stop_range = [11, 17]

In [ ]:
fm_dict[126].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[126].set_filter(route_names=['43'])

In [ ]:
fm_dict[126].corridor_metrics()

In [ ]:
fm_dict[126].quick_map_corridor()

In [ ]:
corr_43 = fm_dict[126].corridor.copy()
corridors += [corr_43]

## 2 - Hotspot Improvements

In [ ]:
fm_dict = bb_rt_helper.fm_from_bbutils(ct_dist='06 - Fresno', category='hotspot', fm_dict = fm_dict)

In [ ]:
hs_speed_views = []
for key in fm_dict.keys():
    _m = fm_dict[key].segment_speed_map()
    hs_speed_views += [fm_dict[key].detailed_map_view.copy()]

## Operator: GET Bus

In [ ]:
m_hs = fm_dict[126].segment_speed_map()

In [ ]:
m_hs

# Draft Hotspot Recs, SDMTS, D11
* SR-99 and Ming Avenue intersection (lots of routes)
* Route 22/46 SR-58 and Chester Ave intersection

## Route 22/42/a bunch of others

In [ ]:
shape_id = '16'
stop_range = [18, 19]

In [ ]:
fm_dict[126].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[126].reset_filter()

In [ ]:
fm_dict[126].corridor_metrics()

In [ ]:
fm_dict[126].quick_map_corridor()

In [ ]:
hs_22 = fm_dict[126].corridor.copy()
hotspots = [hs_22]

## Route 22/46

In [ ]:
shape_id = '16'
stop_range = [14, 15]

In [ ]:
fm_dict[126].autocorridor(shape_id=shape_id, stop_seq_range=stop_range)

In [ ]:
fm_dict[126].reset_filter()

In [ ]:
fm_dict[126].corridor_metrics()

In [ ]:
fm_dict[126].quick_map_corridor()

In [ ]:
hs_22_46 = fm_dict[126].corridor.copy()
hotspots += [hs_22_46]

## Map Testing

In [ ]:
all_speed_segs = pd.concat(hs_speed_views)
all_corr_segs = pd.concat(corr_speed_views)
# full join to preserve all unique segments, since they can differ
# from hotspot to corridor (or indeed across operators)
all_segs = all_speed_segs >> full_join(_, all_corr_segs)

In [ ]:
bb_rt_helper.bb_map_all(hotspots, corridors, all_segs, district = '06 - Fresno')